# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, LSTM, LSTMCell, SimpleRNNCell
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32')  # shape: 60000, 28, 28
x_test = x_test.astype('float32')    # shape: 10000, 28, 28
x_train /= 255  # normalizacja wartości do przedziału [0, 1]
x_test /= 255

y_train = to_categorical(y_train, 10)  # zamiana etykiety na one-hot encoding; np. 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_test = to_categorical(y_test, 10)

11490434/11490434 [==============================] - 2s 0us/step


## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://www.tensorflow.org/guide/keras/rnn

https://www.tensorflow.org/guide/function

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [4]:
class RecurrentModel(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_1 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.lstm_1(inputs)
        return self.dense_1(x)

model = RecurrentModel(num_classes=10)

In [5]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 38s 19ms/step - loss: 0.6452 - accuracy: 0.8065
Epoch 2/2
1875/1875 [==============================] - 33s 17ms/step - loss: 0.1530 - accuracy: 0.9568


### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu, natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w Kerasie zwracają tylko wyniki przetwarzania ostatniego
kroku (otrzymują tensor 3-wymiarowy, zwracają tensor 2-wymiarowy). Jeśli chcesz zwrócić sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej, musisz ustawić parametr return_sequences=True.


In [6]:
class RecurrentModelZad1(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModelZad1, self).__init__(name='my_model_zad1')
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm1 = LSTM(128, activation='relu', return_sequences=True)
        self.lstm2 = LSTM(128, activation='relu')
        self.dense1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        return self.dense1(x)

model = RecurrentModelZad1(num_classes=10)
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 109s 57ms/step - loss: 0.5496 - accuracy: 0.8334
Epoch 2/2
1875/1875 [==============================] - 111s 59ms/step - loss: 0.1501 - accuracy: 0.9632


### Zadanie 2 
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu SimpleRNNCell.

Cell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki SimpleRNNCell (liczba kroków to liczba elementów w sekwencji). Aby wywołać SimpleRNNCell dla pojedynczego wejścia i stanu należy użyć jej metody ```call``` analogicznie jak w przypadku własnych modeli (tzn. ```my_model(input)```). 



Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i **listy macierzy** (w dokumentacji jest błąd, że ma to być macierz) stanów ukrytych poprzedniego kroku (SimpleRNNCell ma jeden stan, LSTMCell w następnym zadaniu ma dwa stany).

Trzeba zainicjalizować ukryty stan warstwy wartościami początkowymi (można wykorzystać rozkład normalny - tf.random.normal).

In [8]:
class RecurrentModelZad2(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModelZad2, self).__init__(name='my_model_zad2')
        self.num_classes = num_classes
        # Define your layers here.
        self.rnn_cell1 = SimpleRNNCell(128, activation='relu')
        self.dense1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        h = [tf.random.normal([inputs.shape[0], 128])]
        for i in range(inputs.shape[1]):
          x, h = self.rnn_cell1(inputs[:,i,:], h)
        return self.dense1(x)

model = RecurrentModelZad2(num_classes=10)
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 11s 5ms/step - loss: 0.4781 - accuracy: 0.8489
Epoch 2/2
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2114 - accuracy: 0.9390


### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell (LSTMCell ma dwa stany ukryte).

In [9]:
class RecurrentModelZad3(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModelZad3, self).__init__(name='my_model_zad3')
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_cell1 = LSTMCell(128, activation='relu')
        self.dense1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        h = [tf.random.normal([inputs.shape[0], 128]), tf.random.normal([inputs.shape[0], 128])]
        for i in range(inputs.shape[1]):
          x, h = self.lstm_cell1(inputs[:,i,:], h)
        return self.dense1(x)

model = RecurrentModelZad3(num_classes=10)
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 25s 12ms/step - loss: 0.6265 - accuracy: 0.8027
Epoch 2/2
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1460 - accuracy: 0.9586


### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wywołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [11]:
class RecurrentModelZad4(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModelZad4, self).__init__(name='my_model_zad4')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        h = tf.random.normal([inputs.shape[0], 128])
        for i in range(inputs.shape[1]):
          h = self.cell(inputs[:,i,:], h)
        return self.dense2(h)

    def cell(self, x, h):
        x = K.concatenate([x, h])
        return self.dense1(x)

model = RecurrentModelZad4(num_classes=10)
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5304 - accuracy: 0.8305
Epoch 2/2
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2317 - accuracy: 0.9333


### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania warstwy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [12]:
class RecurrentModelZad5(Model):
    def __init__(self, num_classes):
        super(RecurrentModelZad5, self).__init__(name='my_model_zad5')
        self.num_classes = num_classes
        self.dense_forget = Dense(128, activation='sigmoid')
        self.dense_input = Dense(128, activation='sigmoid')
        self.dense_gate = Dense(128, activation='tanh')
        self.dense_output = Dense(128, activation='sigmoid')
        self.dense_5 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        h = [tf.random.normal([inputs.shape[0], 128]), tf.random.normal([inputs.shape[0], 128])]
        for i in range(inputs.shape[1]):
          x, h = self.cell(inputs[:,i,:], h)
        return self.dense_5(x)

    def cell(self, x, h):
        c_t_1, h_t_1 = h
        h_x = K.concatenate([h_t_1, x])
        c_t = self.dense_forget(h_x) * c_t_1 + self.dense_input(h_x) * self.dense_gate(h_x)
        h_t = self.dense_output(h_x) * K.tanh(c_t)
        return h_t, (c_t, h_t)

model = RecurrentModelZad5(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 28s 13ms/step - loss: 0.3559 - accuracy: 0.8892
Epoch 2/2
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1184 - accuracy: 0.9644
